In [1]:
import json
import requests

from time import sleep
from urllib.request import urlretrieve

from bs4 import BeautifulSoup

In [2]:
def fetch(url):
    print('fetching {}'.format(url))
    return BeautifulSoup(requests.get(url).content, "html.parser")


def get_imgs(soup):
    divs = soup.findAll('div', class_='vendor-rollout-target')

    if not divs:
        return
    
    results = []
    for d in divs:
        img = d.find('img')
        results.append({
            'title': d.find('a').text,
            'alt': img.get('alt'),
            'size': '{}x{}'.format(img.get('width'), img.get('height')),
            'url': img.get('srcset').split(' ')[0],
            'url2': img.get('src'),
        })

    return results

In [3]:
with open('data/popular-emojis.json') as f:
    data = json.load(f)

data[:5]

[[':outbox_tray:', '📤'],
 [':nauseated_face:', '🤢'],
 [':OK_button:', '🆗'],
 [':raising_hands:', '🙌'],
 [':globe_with_meridians:', '🌐']]

In [4]:
img_data = []
for i, d in enumerate(data[:2]):
    name, emoji = d
    print('{}: {}...'.format(i + 1, name))

    url = 'http://emojipedia.org/search/?q={}'.format(emoji)
    soup = fetch(url)
    imgs = get_imgs(soup)

    if not imgs:
        print('\tno imgs found :(')
        continue

    img_data.append({'name': name, 'emoji': emoji, 'imgs': imgs})

    if i % 5 == 0:
        sleep(1)

1: :outbox_tray:...
fetching http://emojipedia.org/search/?q=📤
2: :nauseated_face:...
fetching http://emojipedia.org/search/?q=🤢


In [5]:
for i, d in enumerate(img_data):
    name_norm = d['name'].replace(':', '').lower()
    tw, = [i for i in d['imgs'] if i['title'] == 'Twitter']
    urlretrieve(tw['url'], 'img/{}.png'.format(name_norm))

    if i % 5 == 0:
        sleep(1)